In [4]:
%matplotlib inline

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
#Reading in the cities csv and saving it to a dataframe
cities_file = "output_data/cities.csv"
cities_file_df = pd.read_csv(cities_file)
cities_file_df.head()

,id,city,country,lat,lng,date,temp,max temp,humidity,cloudiness,wind speed
0,5282297,Bethel,US,41.37,-73.41,1619485945,49.57,51.80,30,1,6.91
1,2126123,Chokurdakh,RU,70.63,147.92,1619485967,25.90,25.90,84,98,2.89
2,4030556,Rikitea,PF,-23.12,-134.97,1619486535,78.10,78.10,72,100,7.29
3,5880054,Barrow,US,71.29,-156.79,1619485925,21.20,21.20,79,1,16.11
4,3471451,Arraial do Cabo,BR,-22.97,-42.03,1619486536,75.06,75.06,78,100,3.78


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [69]:
# set up a parameters dictionary
gmaps.configure(api_key=g_key)

locations = cities_file_df[["lat", "lng"]]
humidity = cities_file_df['humidity']

fig = gmaps.figure()

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False)


fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [58]:
#Creating a new datefram with the below criteria
#max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.

vaction_df = cities_file_df[(cities_file_df['max temp'] > 70) & (cities_file_df['max temp'] < 80) 
                            & (cities_file_df['wind speed'] < 10) & (cities_file_df['cloudiness'] == 0)]
vaction_df

,id,city,country,lat,lng,date,temp,max temp,humidity,cloudiness,wind speed
10,2152659,Port Macquarie,AU,-31.43,152.92,1619486537,73.51,73.99,56,0,1.99
84,108410,Riyadh,SA,24.69,46.72,1619486515,77.00,77.00,22,0,3.44
157,2074865,Carnarvon,AU,-24.87,113.63,1619486574,75.20,75.20,38,0,3.44
245,2161515,Kiama,AU,-34.68,150.87,1619486597,68.92,72.00,60,0,1.99
265,2149475,Singleton,AU,-32.57,151.18,1619486603,68.83,71.01,81,0,1.99
271,1067565,Beloha,MG,-25.17,45.05,1619486365,70.27,70.27,74,0,5.50
312,3347019,Mossamedes,AO,-15.20,12.15,1619486616,74.46,74.46,73,0,7.11
339,1045114,Inhambane,MZ,-23.86,35.38,1619486232,71.20,71.20,88,0,4.85
347,3395981,Maceió,BR,-9.67,-35.74,1619486625,75.20,75.20,88,0,2.30
472,3450144,Santa Luzia,BR,-19.77,-43.85,1619486661,72.23,73.40,64,0,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [59]:
#Creating a new dataframe from the vacations datafram and adding an empty row named "hotel name"
hotel_df = vaction_df.copy()
hotel_df['hotel name'] = ""
hotel_df = hotel_df.reset_index()
hotel_df

,index,id,city,country,lat,lng,date,temp,max temp,humidity,cloudiness,wind speed,hotel name
0,10,2152659,Port Macquarie,AU,-31.43,152.92,1619486537,73.51,73.99,56,0,1.99,
1,84,108410,Riyadh,SA,24.69,46.72,1619486515,77.00,77.00,22,0,3.44,
2,157,2074865,Carnarvon,AU,-24.87,113.63,1619486574,75.20,75.20,38,0,3.44,
3,245,2161515,Kiama,AU,-34.68,150.87,1619486597,68.92,72.00,60,0,1.99,
4,265,2149475,Singleton,AU,-32.57,151.18,1619486603,68.83,71.01,81,0,1.99,
5,271,1067565,Beloha,MG,-25.17,45.05,1619486365,70.27,70.27,74,0,5.50,
6,312,3347019,Mossamedes,AO,-15.20,12.15,1619486616,74.46,74.46,73,0,7.11,
7,339,1045114,Inhambane,MZ,-23.86,35.38,1619486232,71.20,71.20,88,0,4.85,
8,347,3395981,Maceió,BR,-9.67,-35.74,1619486625,75.20,75.20,88,0,2.30,
9,472,3450144,Santa Luzia,BR,-19.77,-43.85,1619486661,72.23,73.40,64,0,4.61,


In [67]:
#Creating search to find nearest hotel to the cities in the hotel dataframe

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "hotel"
target_radius = 5000
target_type = "hotel"

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

#iterating through the dataframe
for index, row in hotel_df.iterrows():
    
    #Getting the lat and lng from the cities in the dataframe and storing them 
    lat = row['lat']
    lng = row['lng']

    #Using the lat and lng coordinates we just stored to set the location params.
    #This is done within the for loop so it will change with each city.
    params["location"] = f"{lat},{lng}"


    #Make request using the base url and params generated
    hotel = requests.get(base_url, params=params)
    
    #Convert to json
    hotel = hotel.json()
     
    #Try and except in case there are no results returned. if no results, enter a nan value    
    try:         
        hotel_df.loc[index, 'hotel name'] = hotel['results'][0]['name']
    except IndexError:
        hotel_df.loc[index, 'hotel name'] = np.nan

In [61]:
#pprint(hotel)

In [68]:
#Drop any nan values that were returned.
hotel_df = hotel_df.dropna()
hotel_df

,index,id,city,country,lat,lng,date,temp,max temp,humidity,cloudiness,wind speed,hotel name
0,10,2152659,Port Macquarie,AU,-31.43,152.92,1619486537,73.51,73.99,56,0,1.99,ibis Styles Port Macquarie
1,84,108410,Riyadh,SA,24.69,46.72,1619486515,77.00,77.00,22,0,3.44,Four Seasons Hotel Riyadh At Kingdom Center
2,157,2074865,Carnarvon,AU,-24.87,113.63,1619486574,75.20,75.20,38,0,3.44,Hospitality Carnarvon
3,245,2161515,Kiama,AU,-34.68,150.87,1619486597,68.92,72.00,60,0,1.99,The Sebel Kiama Harbourside
4,265,2149475,Singleton,AU,-32.57,151.18,1619486603,68.83,71.01,81,0,1.99,Mid City Motor Inn
6,312,3347019,Mossamedes,AO,-15.20,12.15,1619486616,74.46,74.46,73,0,7.11,Hotel Chik Chik Namibe
7,339,1045114,Inhambane,MZ,-23.86,35.38,1619486232,71.20,71.20,88,0,4.85,Sentidos Beach Retreat
8,347,3395981,Maceió,BR,-9.67,-35.74,1619486625,75.20,75.20,88,0,2.30,Meridiano Hotel
9,472,3450144,Santa Luzia,BR,-19.77,-43.85,1619486661,72.23,73.40,64,0,4.61,Hotel Casa Nova


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [72]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heatmap_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))